In [ ]:
#Importing the dataset 
import numpy as np 
import pandas as pd 
import os     #os module is used here for fetching the content of directory containing dataset
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [ ]:
#All the libraries/modules required to build face mask detection model
from bs4 import BeautifulSoup   #Beautiful soup for web scraping
import matplotlib.pyplot as plt
import os    
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout,BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam  
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import cv2


# **DATA PREPROCESSING**

In [ ]:
#Here this function is used to locate the face in the annotation provided to make predictions.
def generate_box(obj):                  
    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text)
    xmax = int(obj.find('xmax').text)
    ymax = int(obj.find('ymax').text)
    
    return [xmin, ymin, xmax, ymax]   #coordinates of Face in  annotation (Just like Bounding Box)
#This function will convert categorical labels into numbers which are undertsandable by the model.
def generate_label(obj):  #generate_label function encodes the three classes and converts the labels into numbers.

    if obj.find('name').text == "with_mask":
        return 1
    elif obj.find('name').text == "mask_weared_incorrect":
        return 2
    return 0
#Using this generate_target function we parse the annotations file and get the objects out from them
def generate_target(image_id, file): 
    with open(file) as f:
        data = f.read()       #we are reading the annotation file here in order to obtain the object under the object tag in annotation xml file.
        soup = BeautifulSoup(data, 'xml')   #BeautifulSoup is used for scraping the annotation file which is in xml format. Here soup is the object of BeautifulSoup
        objects = soup.find_all('object')   #find_all finds all the objects in annotation file for which the function is being called in the form of a list of strings.

        num_objs = len(objects)    #num_objs is an integer variable containing the total number of objects in an image.

        boxes = []
        labels = []     
        for i in objects:
            boxes.append(generate_box(i)) 
            #Now  the face coordinates of objects are being appended to box list
            labels.append(generate_label(i)) #And the associated labels(i.e masked ,not properly masked or Not masked )to labels (list).
            
        boxes=np.array(boxes)           #to convert the boxes(datatype:list) and labels(datatype:list) into numpy array which is accepted by a model.
        labels=np.array(labels)         
        img_id = np.array(image_id)     #img_id is the index of the image  in the dataset 
        target = {}
        target["boxes"] = boxes    #target is a dictionary having key as image array and values as the associated labels
        target["labels"] = labels
        
        return (target,num_objs)
    '''
    -->so basically this generate_target function has the following parameters:
       *image_id: It is the index/number of image in the image folder of dataset.
       *file: It is the path of the annotation file .
    -->This function returns target which is a dictionary:{key:value}
       * Here key :boxes(i.e face coordinates which is the region of interest in the whole image )
       * Here value:labels associated with 
    
    '''

In [ ]:
imgs = list(sorted(os.listdir("/kaggle/input/face-mask-detection/images/")))
len(imgs)     #there are in total 853 images in image folder

In [ ]:
labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))
len(labels)           #853 annotations

In [ ]:
# Here we use the above functions and save results in lists
targets=[]     #stores face coordinates
numobjs=[]     #stores number of faces in each image
#run the loop for number of images we have
for i in range(0,853):
    file_image = 'maksssksksss'+ str(i) + '.png'
    file_label = 'maksssksksss'+ str(i) + '.xml'
    img_path = os.path.join("/kaggle/input/face-mask-detection/images/", file_image)  #it concatenates the path contents.
    label_path = os.path.join("/kaggle/input/face-mask-detection/annotations/", file_label)
    #Generate Label
    target,numobj = generate_target(i, label_path)   
    targets.append(target)
    numobjs.append(numobj)

In [ ]:
type(target)

In [ ]:
type(numobj)

In [ ]:
numobjs[0]   #there are three faces in image 0 for which labeling has been done 

In [ ]:
print(targets[:5])    #targets is a list which contains all the face coordinates and labels of all objects in each and every annotation fil

In [ ]:
# length of target is 853
print(targets[0])    #There are in total three faces in the first image out of which two are not wearing a mask and the third person is wearing a mask properly.  
print(type(targets))     

In [ ]:
import cv2     #For computer vision.
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array    #for converting image format to numpy array format which is acceptible by the model
from tensorflow import keras
face_images=[]
face_labels=[]
for i in range(853):
    img_path = r"../input/face-mask-detection/images/maksssksksss{}.png".format(i)   #this will give the following image path:/input/face-mask-detection/images/maksssksksssi.png
    #read image from specified file path
    img = cv2.imread(img_path)  
    for j in range(numobjs[i]):
#       get coordinates of ith image in list 
        locs=(targets[i]['boxes'][j])
#     Get the face from the image using the coordinates
#the arguments are as ymin , ymax and xmin xmax
        img1=img[locs[1]:locs[3],locs[0]:locs[2]]    # 0-->xmin , 1-->ymin , 2-->xmax , 3-->ymax   (with the help of this command the face is obtained in img1)
        img1 = cv2.resize(img1, (224, 224))   # here the face obtained is resized to (224,224) pixels
        img1 = img_to_array(img1)      #Now the image so obtained is converted into array.
        #img1 = preprocess_input(img1) 
        face_images.append(img1)   # we have our required image in array form 
        face_labels.append(targets[i]['labels'][j])   #and these are the labels that are associated with it

face_images= np.array(face_images, dtype="float32")
face_labels = np.array(face_labels)
'''cv2.imread() method loads an image from the specified file. 
If the image cannot be read (because of missing file, improper permissions, unsupported or invalid format) 
then this method returns an empty matrix.'''

In [ ]:
len(face_images)   #the no. of faceImages and the labels associated with it are the same

In [ ]:
len(face_labels)

In [ ]:
#For counting the datapoints lying in each class.
unique, counts = np.unique(face_labels, return_counts=True)    
dict(zip(unique, counts))   


In [ ]:
#Encode the labels in one hot encode form 
lb = LabelEncoder()
labels = lb.fit_transform(face_labels)
labels = to_categorical(labels)    
labels

DATA AUGMENTATION

In [ ]:
#Perform data augmentation to increase the data for training the model.   #Here aug is the object of ImageDataGenerator.
aug = ImageDataGenerator(
    zoom_range=0.1,
    rotation_range=25,   
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=True,     
    fill_mode="nearest"    #the most imp. parameter of ImageDataGenerator
    )

In [ ]:
INIT_LR = 1e-4     #this is the learning rate .Default learning rate is 0.001 for Adam.
EPOCHS = 20        #The model will run for 20 times
BS = 32            #Standard batch size is 32.

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
import matplotlib.patches as patches
import tensorflow as tf
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential

# SPLITTING DATA INTO TRAIN &TEST SET

In [ ]:
face_images

In [ ]:
#divide data into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(face_images, labels,
	test_size=0.2, stratify=labels, random_state=42)

Normalize the training and testing data to get better results

In [ ]:
# Dividingthe  train and test images by the maximum value (normalize it)
trainX = trainX / 255.0
testX = testX / 255.0

# The min and max values of the training data
trainX.min(), trainX.max()

In [ ]:
trainX.shape

In [ ]:
trainY.shape

In [ ]:
trainX[0].shape

In [ ]:
trainY[0].shape

In [ ]:
del targets,face_images,face_labels     #as we have no use for them

# BUILDING  A MODEL

In [ ]:
# Set random seed
tf.random.set_seed(42)    
#Relu: if input is negative it returns a zero and if input is positive it gives the same element(i.e without any modification)
model_1 = keras.models.Sequential([     #keras Sequential API has been used
    keras.layers.Conv2D(filters = 5, kernel_size = 3, activation = 'relu', 
                        input_shape = (224,224,3)),    #filter : 5 (randomly alloted)
    keras.layers.Conv2D(filters = 5, kernel_size = 3, activation = 'relu'),
    keras.layers.MaxPool2D(pool_size = 2, padding = 'valid'),   #to reduce the dimension of feature map MaxPool is used which will chose the highest of element from the region of the feature map covered by the filter.
    keras.layers.Conv2D(filters = 5, kernel_size = 3, activation = 'relu'),
    keras.layers.Conv2D(filters = 5, kernel_size = 3, activation = 'relu'),
    keras.layers.MaxPool2D(pool_size = 2, padding = 'valid'),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 3, activation = 'softmax')   #units =3 because 3 classes are there: 1.Masked
                                                            #                                      2.with mask but  incorrectluy
])                                                          #                                      3.without mask


'''
The flatten layer is used so as to convert our final feature map(after pooling) into a linear 1-D array
as  the next layer  is the dense layer 
which accepts 1 D tensors only.
'''

# COMPILING AND FITTING A MODEL

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
#helps in optimization and generalization.
model_1.compile(loss="categorical_crossentropy", optimizer=opt,     #as this is a multi-class classification problem thats why we have used categorical crossentropy.
	metrics=["accuracy"])
H = model_1.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY), 
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

#Class having high weight will be considered more importat while training

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model_1, show_shapes=True)

In [ ]:
model_1.summary()

**IMPROVING THE MODEL (TWEAKING THE PREVIOUS MODEL TO GET BETTER ACCURACY)**

In [ ]:
# Set random seed
tf.random.set_seed(42)

model_2 = keras.models.Sequential([
    keras.layers.Conv2D(filters = 10, kernel_size = 3, activation = 'relu', 
                        input_shape = (224,224,3)),
    keras.layers.Conv2D(filters = 10, kernel_size = 3, activation = 'relu'),
    keras.layers.MaxPool2D(pool_size = 2, padding = 'valid'),
    keras.layers.Conv2D(filters = 10, kernel_size = 3, activation = 'relu'),
    keras.layers.Conv2D(filters = 10,  kernel_size = 3, activation = 'relu'),
    keras.layers.MaxPool2D(pool_size = 2, padding = 'valid'),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 3, activation = 'softmax')  
])

In [ ]:
EPOCHS=30

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)    #decay rate is being used here to reduce the learning rate with each epoch.
model_2.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
H = model_2.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,   #it is the number of steps to yield before declaring one epoch finished and starting a new one.
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS) 

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model_2, show_shapes=True)

In [ ]:
model_2.summary()

# PREDICTIONS

In [ ]:
'''Our model outputs a list of prediction probabilities meaning,
it outputs a number for how likely it thinks a particular class is to be the label.
The higher the number in the prediction probabilities list, the more likely the 
model believes that is the right class.'''
ProbsY = model_1.predict(testX)
print(ProbsY)

In [ ]:
# Converting all of the predictions from probabilities to labels
PredictionY = ProbsY.argmax(axis=1)    

# View the first 10 prediction labels
PredictionY[:10]

In [ ]:
#These are the classes
class_names=["without_mask","with_mask","mask_weared_incorrect"]   

In [ ]:
# See the predicted class number and associated label
ProbsY[20].argmax(), class_names[ProbsY[20].argmax()]

MODEL_1

In [ ]:
print("[INFO] evaluating network...")
predIdxs = model_1.predict(testX, batch_size=32)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs
	))

# # serialize the model to disk
# print("[INFO] saving mask detector model...")

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

MODEL_2

In [ ]:
print("[INFO] evaluating network...")
predIdxs = model_2.predict(testX, batch_size=32)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs
	))

# # serialize the model to disk
# print("[INFO] saving mask detector model...")

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()